# PySCES Basic Workflow

This notebook demonstrates the basic workflow for using PySCES to analyze single-cell data.

In [ ]:
# Import packages
import pysces
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
plt.style.use('seaborn-whitegrid')
sns.set_context('notebook')

## 1. Load Data

PySCES can load data from various sources, including CELLxGENE Census, AnnData files, and CSV files.

In [ ]:
# Load data from CELLxGENE Census
# Note: This will download data from the Census, which may take some time
# For this example, we'll limit to a small subset of cells
adata = pysces.read_census(
    "homo_sapiens",
    obs_query="tissue_general == 'blood' and is_primary_data == True",
    max_cells=5000
)

# Alternatively, load from a local file
# adata = pysces.read_anndata("path/to/data.h5ad")

# Print basic information about the dataset
print(f"Dataset shape: {adata.shape}")
print(f"Number of cells: {adata.n_obs}")
print(f"Number of genes: {adata.n_vars}")

## 2. Preprocess Data

Preprocessing includes quality control, filtering, normalization, and transformation.

In [ ]:
# Preprocess data
adata = pysces.preprocess_data(
    adata,
    min_genes=200,
    min_cells=10,
    max_pct_mito=20,
    normalize=True
)

# Apply rank transformation (required for ARACNe)
adata = pysces.rank_transform(adata, target_layer='rank')

# Print updated information
print(f"After preprocessing: {adata.shape}")
print(f"Available layers: {list(adata.layers.keys())}")

## 3. Run ARACNe

ARACNe infers gene regulatory networks from expression data.

In [ ]:
# Load a list of transcription factors
# For this example, we'll use a small subset of TFs
# In practice, you would use a comprehensive list
example_tfs = [
    "STAT1", "STAT3", "IRF1", "IRF8", "GATA1", "GATA2", "GATA3",
    "SPI1", "CEBPA", "CEBPB", "RUNX1", "RUNX2", "RUNX3",
    "MYC", "MYCN", "FOXP3", "FOXO1", "FOXO3", "PAX5", "EBF1"
]

# Filter to TFs that are in our dataset
tf_list = [tf for tf in example_tfs if tf in adata.var_names]
print(f"Using {len(tf_list)} transcription factors")

# Run ARACNe
aracne = pysces.ARACNe(p_value=1e-7, bootstraps=10)  # Use more bootstraps in practice
network = aracne.run(adata, tf_list=tf_list, layer='rank')

# Convert network to regulons
regulons = pysces.aracne_to_regulons(network)

# Print information about the regulons
print(f"Number of regulons: {len(regulons)}")
for i, regulon in enumerate(regulons[:5]):
    print(f"Regulon {i+1}: {regulon['tf']} with {len(regulon['targets'])} targets")

## 4. Run VIPER

VIPER infers protein activity from gene expression and regulons.

In [ ]:
# Run VIPER
activity = pysces.viper(adata, regulons, layer='rank')

# Print information about the activity matrix
print(f"Activity matrix shape: {activity.shape}")
print(f"Top 5 proteins: {activity.index[:5].tolist()}")

## 5. Cluster Cells

Cluster cells based on protein activity profiles.

In [ ]:
# Cluster cells based on protein activity
adata = pysces.cluster_activity(
    adata,
    activity,
    method='leiden',
    resolution=1.0
)

# Print cluster information
print("Cluster sizes:")
print(adata.obs['activity_clusters'].value_counts())

## 6. Identify Master Regulators

Identify master regulators for each cluster.

In [ ]:
# Identify master regulators for each cluster
cluster_mrs = pysces.cluster_mrs(activity, adata.obs['activity_clusters'])

# Print top master regulators for each cluster
for cluster, mrs in cluster_mrs.items():
    print(f"\nTop 5 master regulators for cluster {cluster}:")
    print(mrs[['tf', 'mean_diff', 'adj_p_value']].head(5))

## 7. Visualize Results

Visualize protein activity and master regulators.

In [ ]:
# Plot UMAP with clusters
plt.figure(figsize=(10, 8))
plt.scatter(
    adata.obsm['X_umap'][:, 0],
    adata.obsm['X_umap'][:, 1],
    c=adata.obs['activity_clusters'].astype('category').cat.codes,
    cmap='tab20',
    s=10,
    alpha=0.7
)
plt.colorbar(label='Cluster')
plt.title('UMAP of Protein Activity Clusters')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.show()

In [ ]:
# Plot activity of top master regulators on UMAP
# Get top MRs from each cluster
top_mrs = set()
for cluster, mrs in cluster_mrs.items():
    top_mrs.update(mrs['tf'].head(3).tolist())

# Plot activity
fig = pysces.plot_activity_umap(adata, activity, list(top_mrs), cluster_key='activity_clusters')
plt.show()

In [ ]:
# Plot heatmap of protein activity across clusters
fig = pysces.plot_activity_heatmap(activity, adata.obs['activity_clusters'])
plt.show()

In [ ]:
# Plot master regulators for a specific cluster
cluster_id = adata.obs['activity_clusters'].value_counts().index[0]  # Use the largest cluster
fig = pysces.plot_master_regulators(cluster_mrs[cluster_id])
plt.title(f'Master Regulators for Cluster {cluster_id}')
plt.show()

## 8. Save Results

Save the results for future use.

In [ ]:
# Save AnnData object
# adata.write('results.h5ad')

# Save activity matrix
# activity.to_csv('activity_matrix.csv')

# Save master regulators
# for cluster, mrs in cluster_mrs.items():
#     mrs.to_csv(f'mrs_cluster_{cluster}.csv')